# Tokenizacja za pomocą TF Text

## Przegląd

Tokenizacja to proces dzielenia ciągu znaków na tokeny. Zazwyczaj tokeny te to słowa, liczby i/lub znaki interpunkcyjne. Pakiet `tensorflow_text` udostępnia szereg tokenizerów do wstępnego przetwarzania tekstu wymaganego przez modele tekstowe. Wykonując tokenizację w grafie TensorFlow, nie trzeba martwić się o różnice między przepływami pracy uczenia i wnioskowania oraz zarządzaniem skryptami wstępnego przetwarzania.

W tym Notebook-u omówiono wiele opcji tokenizacji udostępnianych przez TensorFlow Text, sytuacje, w których warto użyć jednej opcji zamiast drugiej oraz sposób wywoływania tokenizatorów z poziomu modelu.

## Konfiguracja

In [1]:
!pip install -q "tensorflow-text==2.11.*"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 86.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 72.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 99.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 99.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 67.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-datasets 4.9.3 requires protobuf>=3.20, but you have protobuf 3.19.6 which is incompatible.
tensorflow-metadata 1.14.0 requires protobuf<4.21,>=3.20.3, but you have protob

In [2]:
import requests
import tensorflow as tf
import tensorflow_text as tf_text

## Splitter API

Głównymi interfejsami są `Splitter` i `SplitterWithOffsets`, które posiadają pojedyncze metody `split` i `split_with_offsets`. Wariant `SplitterWithOffsets` (który rozszerza `Splitter`) zawiera opcję pobierania przesunięć bajtów. Pozwala to wywołującemu dowiedzieć się, z których bajtów oryginalnego łańcucha został utworzony token.

`Tokenizer` i `TokenizerWithOffsets` są wyspecjalizowanymi wersjami `Splitter`, które dostarczają wygodnych metod odpowiednio `tokenize` i `tokenize_with_offsets`.

Ogólnie rzecz biorąc, dla dowolnego N-wymiarowego wejścia, zwrócone tokeny są w N+1-wymiarowym [RaggedTensor](https://www.tensorflow.org/guide/ragged_tensor) z najbardziej wewnętrznym wymiarem tokenów mapującym do oryginalnych pojedynczych łańcuchów.

```python
class Splitter {
  @abstractmethod
  def split(self, input)
}

class SplitterWithOffsets(Splitter) {
  @abstractmethod
  def split_with_offsets(self, input)
}
```

Istnieje również interfejs `Detokenizer`. Każdy tokenizer implementujący ten interfejs może zaakceptować N-wymiarowy poszarpany tensor tokenów i zwykle zwraca N-1-wymiarowy tensor lub poszarpany tensor, który ma podane tokeny złożone razem.

```python
class Detokenizer {
  @abstractmethod
  def detokenize(self, input)
}
```

## Tokenizatory

Poniżej znajduje się zestaw tokenizerów dostarczanych przez TensorFlow Text. Zakłada się, że ciągi wejściowe są w formacie UTF-8.

### Tokenizatory całych słów

These tokenizers attempt to split a string by words, and is the most intuitive way to split text.


#### WhitespaceTokenizer

Text.WhitespaceTokenizer` jest najbardziej podstawowym tokenizerem, który rozdziela ciągi znaków na zdefiniowane przez ICU białe znaki (np. spacja, tabulator, nowa linia). Jest to często dobre rozwiązanie do szybkiego budowania prototypowych modeli.

In [3]:
tokenizer = tf_text.WhitespaceTokenizer()
tokens = tokenizer.tokenize(["What you know you can't explain, but you feel it."])
print(tokens.to_list())

[[b'What', b'you', b'know', b'you', b"can't", b'explain,', b'but', b'you', b'feel', b'it.']]


Można zauważyć, że wadą tego tokenizera jest to, że interpunkcja jest dołączana do słowa, aby utworzyć token. Aby rozdzielić słowa i interpunkcję na osobne tokeny, należy użyć `UnicodeScriptTokenizer`.

#### UnicodeScriptTokenizer

Funkcja `UnicodeScriptTokenizer` dzieli ciągi znaków w oparciu o granice skryptów Unicode. Używane kody skryptów odpowiadają wartościom UScriptCode International Components for Unicode (ICU). ( http://icu-project.org/apiref/icu4c/uscript_8h.html)

W praktyce jest to podobne do `WhitespaceTokenizer` z najbardziej widoczną różnicą polegającą na tym, że rozdziela interpunkcję (USCRIPT_COMMON) od tekstów językowych (np. USCRIPT_LATIN, USCRIPT_CYRILLIC, itp.), jednocześnie oddzielając teksty językowe od siebie. Należy pamiętać, że spowoduje to również rozdzielenie wyrazów skracanych na osobne tokeny.

In [4]:
tokenizer = tf_text.UnicodeScriptTokenizer()
tokens = tokenizer.tokenize(["What you know you can't explain, but you feel it."])
print(tokens.to_list())

[[b'What', b'you', b'know', b'you', b'can', b"'", b't', b'explain', b',', b'but', b'you', b'feel', b'it', b'.']]


### Tokenizatory podsłów

Subword tokenizers can be used with a smaller vocabulary, and allow the model to have some information about novel words from the subwords that make create it.



#### WordpieceTokenizer

Tokenizacja WordPiece to oparty na danych schemat tokenizacji, który generuje zestaw tokenów podrzędnych.

WordpieceTokenizer oczekuje, że dane wejściowe są już podzielone na tokeny. Z powodu tego warunku wstępnego, często będziesz chciał wcześniej dokonać podziału za pomocą `WhitespaceTokenizer` lub `UnicodeScriptTokenizer`.

In [5]:
tokenizer = tf_text.WhitespaceTokenizer()
tokens = tokenizer.tokenize(["What you know you can't explain, but you feel it."])
print(tokens.to_list())

[[b'What', b'you', b'know', b'you', b"can't", b'explain,', b'but', b'you', b'feel', b'it.']]


Po podzieleniu łańcucha na tokeny, `WordpieceTokenizer` może zostać użyty do podzielenia go na podtokeny.

In [6]:
url = "https://github.com/tensorflow/text/blob/master/tensorflow_text/python/ops/test_data/test_wp_en_vocab.txt?raw=true"
r = requests.get(url)
filepath = "vocab.txt"
open(filepath, 'wb').write(r.content)

52382

In [7]:
subtokenizer = tf_text.UnicodeScriptTokenizer(filepath)
subtokens = tokenizer.tokenize(tokens)
print(subtokens.to_list())

[[[b'What'], [b'you'], [b'know'], [b'you'], [b"can't"], [b'explain,'], [b'but'], [b'you'], [b'feel'], [b'it.']]]


#### BertTokenizer

BertTokenizer odzwierciedla oryginalną implementację tokenizacji z artykułu BERT. Jest on wspierany przez WordpieceTokenizer, ale wykonuje również dodatkowe zadania, takie jak normalizacja i tokenizacja słów.

In [8]:
tokenizer = tf_text.BertTokenizer(filepath, token_out_type=tf.string, lower_case=True)
tokens = tokenizer.tokenize(["What you know you can't explain, but you feel it."])
print(tokens.to_list())

[[[b'what'], [b'you'], [b'know'], [b'you'], [b'can'], [b"'"], [b't'], [b'explain'], [b','], [b'but'], [b'you'], [b'feel'], [b'it'], [b'.']]]


#### SentencepieceTokenizer

SentencepieceTokenizer to tokenizator sub-tokenów, który jest wysoce konfigurowalny. Jest on wspierany przez bibliotekę Sentencepiece. Podobnie jak BertTokenizer, może obejmować normalizację i podział tokenów przed podziałem na tokeny podrzędne.


In [9]:
url = "https://github.com/tensorflow/text/blob/master/tensorflow_text/python/ops/test_data/test_oss_model.model?raw=true"
sp_model = requests.get(url).content

In [10]:
tokenizer = tf_text.SentencepieceTokenizer(sp_model, out_type=tf.string)
tokens = tokenizer.tokenize(["What you know you can't explain, but you feel it."])
print(tokens.to_list())

[[b'\xe2\x96\x81What', b'\xe2\x96\x81you', b'\xe2\x96\x81know', b'\xe2\x96\x81you', b'\xe2\x96\x81can', b"'", b't', b'\xe2\x96\x81explain', b',', b'\xe2\x96\x81but', b'\xe2\x96\x81you', b'\xe2\x96\x81feel', b'\xe2\x96\x81it', b'.']]


### Inne splitters


#### UnicodeCharTokenizer

Dzieli ciąg znaków na znaki UTF-8.

In [11]:
tokenizer = tf_text.UnicodeCharTokenizer()
tokens = tokenizer.tokenize(["What you know you can't explain, but you feel it."])
print(tokens.to_list())

[[87, 104, 97, 116, 32, 121, 111, 117, 32, 107, 110, 111, 119, 32, 121, 111, 117, 32, 99, 97, 110, 39, 116, 32, 101, 120, 112, 108, 97, 105, 110, 44, 32, 98, 117, 116, 32, 121, 111, 117, 32, 102, 101, 101, 108, 32, 105, 116, 46]]


Wynikiem są liczby kodowe Unicode. Może to być również przydatne do tworzenia ngramów znaków, takich jak bigramy. Aby przekonwertować z powrotem na znaki UTF-8.

In [12]:
characters = tf.strings.unicode_encode(tf.expand_dims(tokens, -1), "UTF-8")
bigrams = tf_text.ngrams(characters, 2, reduction_type=tf_text.Reduction.STRING_JOIN, string_separator='')
print(bigrams.to_list())

[[b'Wh', b'ha', b'at', b't ', b' y', b'yo', b'ou', b'u ', b' k', b'kn', b'no', b'ow', b'w ', b' y', b'yo', b'ou', b'u ', b' c', b'ca', b'an', b"n'", b"'t", b't ', b' e', b'ex', b'xp', b'pl', b'la', b'ai', b'in', b'n,', b', ', b' b', b'bu', b'ut', b't ', b' y', b'yo', b'ou', b'u ', b' f', b'fe', b'ee', b'el', b'l ', b' i', b'it', b't.']]


#### RegexSplitter

RegexSplitter jest w stanie segmentować ciągi znaków w dowolnych punktach przerwania zdefiniowanych przez podane wyrażenie regularne.

In [14]:
splitter = tf_text.RegexSplitter("\s")
tokens = splitter.split(["What you know you can't explain, but you feel it."], )
print(tokens.to_list())

[[b'What', b'you', b'know', b'you', b"can't", b'explain,', b'but', b'you', b'feel', b'it.']]


## Offsets

Podczas tokenizacji ciągów znaków, często pożądane jest, aby wiedzieć, skąd w oryginalnym ciągu znaków pochodzi token. Z tego powodu każdy tokenizator, który implementuje `TokenizerWithOffsets` posiada metodę *tokenize_with_offsets*, która zwróci przesunięcia bajtów wraz z tokenami. Start_offsets zawiera listę bajtów w oryginalnym łańcuchu, od których zaczyna się każdy token, a end_offsets zawiera listę bajtów bezpośrednio po punkcie, w którym kończy się każdy token. Aby uściślić, przesunięcia początkowe są włącznie, a przesunięcia końcowe są wyłączne.

In [15]:
tokenizer = tf_text.UnicodeScriptTokenizer()
(tokens, start_offsets, end_offsets) = tokenizer.tokenize_with_offsets(['Everything not saved will be lost.'])
print(tokens.to_list())
print(start_offsets.to_list())
print(end_offsets.to_list())

[[b'Everything', b'not', b'saved', b'will', b'be', b'lost', b'.']]
[[0, 11, 15, 21, 26, 29, 33]]
[[10, 14, 20, 25, 28, 33, 34]]


## Detokenization

Tokenizery implementujące `Detokenizer` zapewniają metodę `detokenize`, która próbuje połączyć ciągi. Może to być stratne, więc zdetokenizowany ciąg może nie zawsze dokładnie odpowiadać oryginalnemu, wstępnie ztokenizowanemu ciągowi.

In [16]:
tokenizer = tf_text.UnicodeCharTokenizer()
tokens = tokenizer.tokenize(["What you know you can't explain, but you feel it."])
print(tokens.to_list())
strings = tokenizer.detokenize(tokens)
print(strings.numpy())

[[87, 104, 97, 116, 32, 121, 111, 117, 32, 107, 110, 111, 119, 32, 121, 111, 117, 32, 99, 97, 110, 39, 116, 32, 101, 120, 112, 108, 97, 105, 110, 44, 32, 98, 117, 116, 32, 121, 111, 117, 32, 102, 101, 101, 108, 32, 105, 116, 46]]
[b"What you know you can't explain, but you feel it."]


## TF Data

TF Data to potężny interfejs API do tworzenia potoku danych wejściowych do trenowania modeli. Tokenizery działają zgodnie z oczekiwaniami z API.

In [17]:
docs = tf.data.Dataset.from_tensor_slices([['Never tell me the odds.'], ["It's a trap!"]])
tokenizer = tf_text.WhitespaceTokenizer()
tokenized_docs = docs.map(lambda x: tokenizer.tokenize(x))
iterator = iter(tokenized_docs)
print(next(iterator).to_list())
print(next(iterator).to_list())

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


[[b'Never', b'tell', b'me', b'the', b'odds.']]
[[b"It's", b'a', b'trap!']]
